# cuQuantum-Appliance: Google Cirq accelerated with cuQuantum

In this tutorial, we'll learn how to simulate quantum circuits constructed with Google Cirq on GPUs using cuQuantum-accelerated qsim.

_Cirq_ is a popular python framework for programming quantum computers circuit by Google's QuantumAI team. It provides an extensive set of gates, algorithms and examples and can execute quantum algorithms either on simulator backends or quantum hardware.

_qsim_ is Google's fast state vector simulator for Cirq. For fast GPU simulations it uses the NVIDIA cuQuantum SDK.

The _NVIDIA cuQuantum Appliance_ docker image provides a ready-to-go image containing all components and allows GPU-accelerated simulations on all NVIDIA platforms with Volta-architecture or newer.

More information on the different components can be found on:
* NVIDIA cuQuantum Appliance: https://catalog.ngc.nvidia.com/orgs/nvidia/containers/cuquantum-appliance
* NVIDIA cuQuantum SDK: https://developer.nvidia.com/cuquantum-sdk
* Google Cirq: https://quantumai.google/cirq
* Google qsim: https://quantumai.google/qsim

## Setup

As first step, we load the Cirq and qsim python modules

In [ ]:
import cirq
import qsimcirq

## Building a circuit

As warmup, we build a circuit to construct a 5-qubit GHZ state. A GHZ state is a non-classical state which entangles all qubits. It is defined as

$|GHZ \rangle = \frac{1}{\sqrt{2}} \left(|00000 \rangle + |11111 \rangle \right)$
    
At first we create a new quantum circuit in Cirq and a list of 5 qubits. 

In [ ]:
c = cirq.Circuit()
qubits = cirq.LineQubit.range(5)

We assume all qubits will be in state $0$, i.e., the system is in state $|00000\rangle$.

In our circuit, we first apply a Hadamard gate on qubit 0 putting it into state
$ |\psi_0 \rangle = \frac{1}{\sqrt{2}} \left( |0 \rangle + | 1 \rangle \right) $
and entangle all qubits with qubit 0 by applying a chain controlled-NOT gates.

In [ ]:
c.append(cirq.H(qubits[0]))
c.append(cirq.CNOT(qubits[0], qubits[1]))
c.append(cirq.CNOT(qubits[1], qubits[2]))
c.append(cirq.CNOT(qubits[2], qubits[3]))
c.append(cirq.CNOT(qubits[3], qubits[4]))

Cirq allows us to print our circuit:

In [ ]:
print(c)

Using Cirq's internal simulator we can easily simulate this small circuit and verify that the resulting quantum state is indeed a GHZ state.

In [ ]:
s = cirq.Simulator()
s.simulate(c)

If we add a measurement to our circuit and sampling the (classical) outcome several times, we can also observe the effect of the entanglement:
the system will be either in a state where all measured qubits are $0$ or all qubits are $1$. We never observe a situation in which only a few of the measured qubits in state $1$ and the remaining in state $0$.

In [ ]:
c.append(cirq.measure_each(*qubits))
s.sample(c, repetitions=10)

## Running a more complex circuit

A common building block of many quantum algorithms is the Quantum Fourier Transform (QFT), which transforms the basis states of an N-qubit system an integer $|n\rangle$ 
to a Fourier basis

$$
\text{QFT}|n\rangle = \frac{1}{2^N} \sum_{k=0}^{n-1} \omega_{2^N}^{nk} |k\rangle
$$

where $\omega_{2^N} = e^{2 \pi i / 2^N}$. This algorithm is already provided by Cirq and can be used directly in a circuit:

In [ ]:
c = cirq.Circuit()
qubits = cirq.LineQubit.range(22)
c.append(cirq.qft(*qubits))
c.append(cirq.measure_each(*qubits))

Simulating this circuit with Cirq's native simulator is a bit time consuming...

In [ ]:
import time
s = cirq.Simulator()
start = time.time()
r = s.sample(c, repetitions=20)
print(r)
print("runtime: {:.2f}s".format(time.time() - start))

## Running with qsim + cuQuantum

The same circuit simulated with qsim on GPUs using NVIDIA cuQuantum library can accelerate this simulation substantially.

We first need to setup the simulator with GPU support. The NVIDIA cuQuantum Appliance added a few options to QsimOptions:

#### gpu_mode
The GPU simulator backend to use. If 1, the simulator backend will use cuStateVec. If n, an integer greater than 1, the simulator will use the multi-GPU backend with the first n devices (if available). If a sequence of integers, the simulator will use the multi-GPU backend with devices whose ordinals match the values in the list. Default is to use the multi-GPU backend with device 0

#### use_sampler
If None, the multi-GPU backend will use its sampler, and all other backends will use their default sampler. If True, use the multi-GPU backend’s sampler. If False, the multi-GPU backend’s sampler is disabled

#### disable_gpu
Whether or not to disable the GPU simulator backend. All GPU options are only considered when this is False (default). Note the difference from qsimcirq’s use_gpu keyword.

We setup the qsim simulator with one GPU:

In [ ]:
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
        max_fused_gate_size = 2
        , cpu_threads = 1
        , gpu_mode = ngpus
        , use_sampler = True
        , disable_gpu = False
    )
qsim_simulator = qsimcirq.QSimSimulator(qsim_options)

... and rerun the same circuit simulation on the GPU using the same syntax as before

In [ ]:
start = time.time()
r = qsim_simulator.sample(c, repetitions=20)
print(r)
print("runtime: {:.2f}s".format(time.time() - start))

## Factoring Integers

The most famous application of quantum computers is factoring integers using Shor's algorithm. It is of considerate interest because an efficient algorithm for factoring would allow to crack modern asymmetric encryption schemes, such as RSA.

For small integers this quantum algorithm can be simulated on a classical computer.

Here we need a set of additional tools and helper functions, the actual algorithm will follow further down.

In [ ]:
from math import gcd, log2, ceil
import numpy as np
from random import randint
from qiskit.algorithms import Shor
import fractions
import data.qiskit_to_cirq_conversion

In [ ]:
def get_num_bits(N):
    return int(log2(N)+1)

In [ ]:
def get_histogram(result, width, shots):
    if shots == 1:
        return {result.tobytes(): 1}

    # We rely on the PY37+ feature that the dict is ordered, and sort the
    # measurement outcome (significant ones come first)
    out = {}
    axis = None if result.ndim == 1 else 0
    keys, values = np.unique(result, axis=axis, return_counts=True)
    idx = np.argsort(values)[::-1]
    keys = keys[idx]
    values = values[idx]
    for i in range(keys.shape[0]):
        # numpy arrays are not hashable
        if result.ndim == 2:  # from qsimcirq
            k = ''.join(str(keys[i])[1:-1].split())
        else:
            assert False
        out[k] = values[i]

    # sanity check
    counts = 0
    for k, v in out.items():
        counts += v
    assert counts == shots, f"counts = {counts}, shots = {shots}"

    if shots/len(out) < 5.0:
        print(f"WARNING: too many ({len(out)}) unique bitstrings with limited shots ({shots}), "
              "statistics may not be accurate")

    return out


In [ ]:
def get_factors(r, r_nbits, x, N):
    assert r_nbits > 0
    assert x > 0
    assert N > 0
    eigenphase = float(r) / 2**r_nbits
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(N)    
    # If the numerator is zero, the order finder failed.
    if f.numerator == 0:
        return None
    
    # Else, return the denominator if it is valid.
    r = f.denominator
    if x**r % N != 1:
        return None
    return r

### Shor's algorithm
The integer factoring problem is finding the components $g$, $h$ of a given product $N=g \cdot h$ of two primes.

The algorithm consists of two ideas.

##### 1. Reduce the problem of factoring the integer to the order-finding problem (classically):
Given an integer $a$ find the period $x=r$ of $a^x (\text{mod} N)$, <br>
i.e., the smallest positive integer $r$ that satisfies $a^r = 1 (\text{mod} N)$.

In [ ]:
def shors_algorithm(N):
    while(True):
        # 1. Select a random integer between 2 and N-1
        a = randint(2,N-1)
        
        # 2. See if it happens to factor N
        print("Trying a =", a)
        g = gcd(a,N)
        if g != 1:
            print("Found factor by chance:", g)
            return (g, N//g)
        
        # 3. Find the period a^r = 1 (mod N)
        r = find_period(a, N)
        
        # 4. If a period is found and it is
        # * even and
        # * not a^(r/2) = -1 (mod N),
        # the factors will be the gcd(a^(r/2) + 1, N), gcd(a^(r/2 - 1, N).
        if r != None and r % 2 != 1 and (r//2) % N != N-1:
            a1 = gcd(a ** (r//2) + 1, N)
            print("Found factor:", a1)
            return (a1, N//a1)
        print("retrying...")

##### 2. Solve the order-finding problem with a quantum algorithm.

The key component of the algorithm is an efficient quantum algorithm to find the period $r$ of the operation $x^r (\text{mod} N)$. A naive classical algorithm for this problem is easy to implement, but inefficient:

In [ ]:
def find_period(a, N):
    return find_period_classical(a, N)

In [ ]:
def find_period_classical(a, N):
    """A naive classical method to find the period r of x^r (mod N)."""
    assert 1 < a and a < N
    r = 1
    y = a
    while y != 1:
        y = y * a % N
        r += 1
    return r

In [ ]:
my_integer = 3*5
shors_algorithm(my_integer)

An efficient quantum solution derives the period from a measurement of $n = \lceil log2(N) \rceil$ qubits:

In [ ]:
def find_period(a, N):
    return find_period_quantum(a, N)

In [ ]:
def find_period_quantum(x, N):
    """The quantum algorithm to find the period r of x^r (mod N)."""
    
    shots   = 128
    nbits   = get_num_bits(N)
    
    qiskit_circuit              = Shor().construct_circuit(N, x)
    cirq_circuit, qubit_offsets = data.qiskit_to_cirq_conversion.convert(qiskit_circuit)
    print("Total number of qubits:", qubit_offsets['total'])
    cirq_circuit.append(cirq.measure_each(*cirq.LineQubit.range(qubit_offsets['up'], qubit_offsets['down'])))

    result = qsim_simulator.sample(cirq_circuit,repetitions=shots)
    measurement_results = result.to_numpy()
    print("Measurement results:")
    print(measurement_results)
    hist = get_histogram(measurement_results, qubit_offsets['down'], shots)
    print("Number of occurences:", hist)
    most_probable_bitpattern = max(hist, key=hist.get)
    result = int(most_probable_bitpattern[::-1],2)
    print("Most probable bitpattern:", most_probable_bitpattern, "=", result)
    r = get_factors(result, qubit_offsets['down'], x, N)
    print("Found period:", r)
    return r

In [ ]:
my_integer = 3*5
shors_algorithm(my_integer)

# References

* P. Shor, 1997 , "Polynomial-time algorithms for prime factorization and discrete logarithms on a quantum computer", SIAM J. Comp., 26, pp. 1484-150
* S. Beauregard, 2003, "Circuit for Shor's algorithm using 2n+3 qubits.", Quantum Info. Comput. 3, 2 (March 2003), 175–185.